# Mobile platform planning 🏎️

This notebook illustrates how to do motion planning for a mobile platform. We only show how to do motion planning for the platform (3 DOF) without any mounted UR arm (6 DOF). Full platform motion planning (with 9 DOF) is not yet implemented, but should be possible to do with a similar set-up.

## 1. Setup 🏗️️

### 1.1 Building our mobile platform scene

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from pydrake.math import RigidTransform, RollPitchYaw
from airo_drake import MobilePlatformWithSingleArmScene, add_wall, add_mobile_platform, finish_build, add_meshcat, add_manipulator, attach_mobile_platform_to_world

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
mobi_index = add_mobile_platform(robot_diagram_builder, drive_positions=(np.array([1, -0.5]), np.array([1, 0.5]), np.array([-1, -0.5]),
                                                np.array([-1, 0.5])), cpu_position=np.array([0, -0.5]), battery_position=np.array([0, 0.5]))
mobi_frame = robot_diagram_builder.plant().GetFrameByName("base_link", mobi_index)

attach_mobile_platform_to_world(robot_diagram_builder, mobi_index)

# For these two values, see the API of `add_mobile_platform`.
side_height = 0.43
roof_thickness = 0.03
arm_transform = RigidTransform(p=[0.2445, 0, side_height + roof_thickness], rpy=RollPitchYaw([0, 0, -np.pi / 2]))
# We make the arm static, to do motion planning purely with the platform.
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur5e", "robotiq_2f_85", arm_transform, static_arm=True, static_gripper=True, parent_frame=mobi_frame)

# We also add an obstacle to the scene for the platform to avoid.
add_wall(robot_diagram_builder, "YZ", position=np.array([1.0, 0.0, 0.0]))

robot_diagram, context = finish_build(robot_diagram_builder)

scene = MobilePlatformWithSingleArmScene(robot_diagram, mobi_index, arm_index, gripper_index, meshcat)
scene

### 1.2 Mobile platform `SceneGraphCollisionChecker` 💥

We need to provide the model indices for the mobile robot. We only motion plan the mobile platform for now, not everything at once.

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[
        scene.mobile_platform_index,
        scene.arm_index,
        scene.gripper_index
    ],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
scene.robot_diagram.plant().num_positions()

In [ ]:
collision_checker.CheckConfigCollisionFree([1.0, 0.0, 0]), collision_checker.CheckConfigCollisionFree([1.0, 0.0, np.pi/2])

In [ ]:
plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, [1.0, 0.0, np.pi/2])
scene.robot_diagram.ForcedPublish(context)  # updates the meshcat visualization

### 1.3 Inverse kinematics for the mobile robot 🧮

The mobile robot's 3 DOF are represented in Drake with a `PlanarJoint` with no bounds on the `x`, `y` and `theta` values.
This means that the forward and inverse kinematics of the platform can be implemented with identity functions.

In other words, we don't even need to define an IK function!

### 1.4 `MobilePlatformOmplPlanner` 🧭

In [ ]:
from airo_planner import MobilePlatformOmplPlanner

planner = MobilePlatformOmplPlanner(
    is_state_valid_fn=collision_checker.CheckConfigCollisionFree
)

## 2. Moving around 🚶

In this section, we demonstrate the `airo-planner` API by moving around the wall with the planner.

In [ ]:
start_pose = np.array([-1.0, 0, np.pi / 2])
goal_pose = np.array([5.0, 0, 0])

plant.SetPositions(plant_context, start_pose)
scene.robot_diagram.ForcedPublish(context)  # updates the meshcat visualization

In [ ]:
path = planner.plan_to_pose(
    start_pose, goal_pose
)
print(path.shape)

In [ ]:
from airo_drake import time_parametrize_toppra_mobile_platform, animate_mobile_platform_trajectory

trajectory = time_parametrize_toppra_mobile_platform(scene.robot_diagram.plant(), path)

animate_mobile_platform_trajectory(
    scene.meshcat, scene.robot_diagram, scene.mobile_platform_index, trajectory
)